# Update POV antenna power from CSV file

Update antenna power after fine tuning for a PassThrough/XPoint for exampe

CSV file format (separator ";")

|    IP	    | ANT |	POWER |
|-----------|---------|-------|
|10.30.11.11|	1	  |  25   |
|10.30.11.11|	2	  |  25   |

## Process

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

import pandas as pd
import numpy as np

In [ ]:
modeOfDevice = 'XPOINT' # or 'AUTOMATED_PROCESS' 'GUARD'

In [ ]:
filePath = uiUtils.getOneFile(initialdirectory=None, windowTitle = "Select file",filtersDefinition = (("csv files","*.csv"),("all files","*.*")))

In [ ]:
%%capture
myEnv = chooseEnv.choosePremise()

In [ ]:
print("Organization:" + myEnv.get('urlDP') + " Premise code "+  myEnv.get('PremiseCode'))

In [ ]:
%%capture
devices = pd.DataFrame(deviceUtils.getDevicesOfPremise(myEnv))
devices = devices.drop_duplicates(subset=['id'])

In [ ]:
tunning = pd.read_csv(filePath,sep=";",converters={'IP':str,'ANT':str,'POWER':str})
tunning = tunning.drop_duplicates()

In [ ]:
tunning.head()

In [ ]:
# keep reader with ip from the list
devices = devices.loc[devices['address'].isin(tunning['IP'])]

In [ ]:
devices.head()

In [ ]:
%%capture
devicesView = {}
for deviceId in devices['id'].tolist():
    devicesView[deviceId] = deviceUtils.getDeviceById(myEnv, deviceId)

In [ ]:
povs = pd.DataFrame()
povs['readerId'] = None
povs['ip'] = None
povs['mode'] = None
povs['povId'] = None
povs['address'] = None
povs['txPowerinDbm'] = None
for deviceId, deviceView in devicesView.items():
    ip = str(deviceView['device']['address'])
    for mode in  deviceView['deviceMode']:
        mode = mode['id']
        # filter by MODE
        if (mode == modeOfDevice):
            for pov in deviceView['devicePov']:
                povId = pov['id']
                address = (pov['address']) # TODO split address separated by ,
                value = tunning[(tunning.IP == ip) & (tunning.ANT == address)]
                if not value.empty:
                    try :
                        txPower = float( value.iloc[0]['POWER'])
                        povs = povs.append({'readerId' : deviceId, 'ip' : ip, 'mode' : mode, 'povId' :povId, 'address' : address, 'txPowerinDbm' : txPower}, ignore_index=True)
                    except ValueError:
                        print( "POWER: "+value.iloc[0]['POWER'] + " can not be convert to float") 

In [ ]:
povs.head()

In [ ]:
for index, row in povs.iterrows():
    try:
        settingsUtils.updatePovRfidSetting(myEnv, row['povId'], row['mode'], 'txPowerinDbm', row['txPowerinDbm'])
    except Exception as error:
        print("Pov",row['povId'],"for reader",row['readerId'],"not processed because:", error)

## Result

In [ ]:
for index, row in povs.iterrows():
    try:
        v = settingsUtils.getRfidSettingsByPovId(myEnv, row['povId'], row['mode'])
        if v['txPowerinDbm'] == row['txPowerinDbm']:
            print('OK    : ',"Pov",row['povId'],"for reader",row['readerId'])
        else:
            print('FAILED: ',"Pov",row['povId'],"for reader",row['readerId']," - (",v['txPowerinDbm'],'!=',row['txPowerinDbm'],')')
    except Exception as error:
        print('ERROR : ','Pov',row['povId'],'for reader',row['readerId'],' - (',error,')')